In [1]:
import pandas as pd
import numpy as np

# Hela ML libraries 
from hela import hmm
import hela.generation.hmm as hmm_gen

# Viz libraries
import altair as alt
import hela.visualization.hmm as hmmplot 
import matplotlib.pyplot as plt
from hela.visualization.hmm import TU_COLORS
%matplotlib inline

# Utility Libraries
from datetime import datetime
from dask.distributed import Client
from scipy.special import logsumexp
from scipy import stats
import itertools
from IPython.display import Image

In [2]:
gen = hmm_gen.FactoredHMMGenerativeModel(
                                     ns_hidden_states =[4, 4],
                                     n_gaussian_features = 4,
                                     n_categorical_features = 0)

factored_hidden_states = gen.generate_hidden_state_sequence(n_observations = 500)

In [3]:
data = gen.generate_observations(factored_hidden_states)
# fhmm_training_spec = gen.generative_model_to_fhmm_training_spec()
fhmm_training_spec = hmm_gen.data_to_fhmm_training_spec(factored_hidden_states, 
                                   gen.ns_hidden_states, 
                                   data,
#                                    categorical_features = [], 
                                   gaussian_features = list(gen.gaussian_values.columns)
                                                       )

fhmm_config = hmm.FactoredHMMConfiguration.from_spec(fhmm_training_spec)

untrained_model = fhmm_config.to_model()

In [4]:
data.head()

,gaussian_feature_0,gaussian_feature_1,gaussian_feature_2,gaussian_feature_3
2020-08-01,-1.798800,-3.467944,3.644544,-0.471293
2020-08-02,-0.911950,-2.766553,2.856906,1.247256
2020-08-03,0.605806,-3.197247,3.045204,1.142476
2020-08-04,-0.372477,-2.781081,3.031308,0.423955
2020-08-05,-2.623511,-3.873819,3.634506,0.223079


In [5]:
alg = untrained_model.load_learning_interface()
model = alg.run(data = data, 
                 method = 'structured_vi', 
                 training_iterations = 10)

In [6]:
inf = fhmm_model.load_inference_interface(fhmm_data)
h_t = np.zeros((len(fhmm_data), 
                        len(fhmm_model.ns_hidden_states), 
                        np.max(fhmm_model.ns_hidden_states)))

for i in range(5):
    gamma, alpha, beta = inf.log_forward_backward(h_t)
    h_t = inf.h_t_update(gamma, fhmm_data)
log_path = inf.predict_hidden_states_log_viterbi(h_t)

NameError: name 'fhmm_model' is not defined

In [ ]:
flattened_pred_states = gen.flatten_hidden_state_sequence(log_path)
hmmplot.draw_states(flattened_pred_states, hide_brush=False)

In [ ]:
flattened_hidden_states = gen.flatten_hidden_state_sequence(factored_hidden_states)
hmmplot.draw_states(flattened_hidden_states, hide_brush=False)

In [ ]:
hmmplot.draw_states(factored_hidden_states[0], hide_brush=False)

In [ ]:
hmmplot.draw_states(factored_hidden_states[1], hide_brush=False)

In [ ]:
likelihood = []
for m in alg.model_results:
    spec = hmm._factored_hmm_to_discrete_hmm(m)
    hmm_config = hmm.DiscreteHMMConfiguration.from_spec(spec)
    hmm_model = hmm_config.to_model()
    hmm_inf = hmm_model.load_inference_interface()
    log_prob = hmm_inf.predict_hidden_state_log_probability(fhmm_data)
    likelihood.append(
        logsumexp(hmm_inf._compute_forward_probabilities(log_prob)[-1]))
#     print(m.transition_matrix)